# 1. Problem Set 2: Data manipulation with `pandas`

**Total points**: 53

By investigating disparities in sentencing between black defendants and white defendants, this problem set will give you practice with:

- Data wrangling with pandas: groupby, subsetting, sorting, etc.
- Defining your own functions
- Visualizing trends in data

We will use the Cook County, Illinois (which contains Chicago) sentencing dataset. This analysis could be extended to study Hispanic defendants or, in a different jurisdiction, Asian and other minoritized groups.

This dataset reports the sentence given to defendants convicted of different crimes, and you can find [the data codebook here](https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf) and the latest on these data [at the official website](https://datacatalog.cookcountyil.gov/Courts/Sentencing/tg8v-tm6u).

**Details if interested in digging deeper** (optional): There is a lot to think about here in terms of (1) how we might measure disparities, and (2) what factors you would want to adjust for when deciding whether two defendants are 'similarly situated' but for their race. You can read more technical coverage in the following sources:

- [Review of sentencing disparities research](https://www.journals.uchicago.edu/doi/full/10.1086/701505)
- [Discussion of causal model/blinding race at charging stage of the prosecutorial process](https://5harad.com/papers/blind-charging.pdf)
- [Discussion of measuring discrimination in policing that can generalize to the sentencing case](https://www.annualreviews.org/doi/abs/10.1146/annurev-criminol-011518-024731)
- [General discussion of causal challenges in measuring between-group disparities](https://osf.io/preprints/socarxiv/gx4y3/)

**One major caveat**: when comparing whether two similar defendants received different sentences, we're missing one important attribute that influences sentencing: the defendant's criminal history. This influences sentencing both through sentencing guidelines, which can prescribe longer sentences for those who have certain types of prior convictions, and through judicial discretion if judges are more lenient with first-time defendants. The above sources discuss how much we want to "control away" for this prior history, since if we think there are racial biases in which defendants, conditional on *committing* a crime, are arrested and charged, we may not want to adjust for that factor.

# 0. Load packages and imports (2 points total)

In [ ]:
## basic functionality
import pandas as pd
import numpy as np
import re

## for plotting; can also use matplotlib or seaborn
## note: for plotnine, you likely need to install using pip or conda
## if not using plotnine for viz, no need to import it (can comment out next two lines)
import plotnine
from plotnine import *
import matplotlib.pyplot as plt

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## datetime util
from dateutil.relativedelta import relativedelta

## 0.1 Load the raw data (1 point)

Use `pd.read_csv` to load the `sentencing_asof0405.csv` data. To get this, unzip `pset2_inputdata.zip` (try the `unzip` shell command). Be sure to use relative path names (e.g., `../data/file.csv` goes up a level and looks inside the `data/` folder for `file.csv`) and **don't hard code** your user-specific path name (e.g., `C:/files/data/file.csv`). If you can't find the file,  double-check you're looking in the right directory (the `os` library in Python can help with this).

*Notes*: You may receive a warning about mixed data types upon import; feel free to ignore, or call `low_memory=False` as a parameter.

**Hint:**
You may receive a warning about mixed data types upon loading the .csv file into pandas; feel free to ignore, or call `low_memory=False` as a parameter.

In [ ]:
## your code here loading the data

## 0.2 Inspect the data (1 points)

Print the head, dimensions, and info for the data.

In [ ]:
## your code here inspecting the data

# 1. Data cleaning/interpretation (21 points total)

## 1.1: Understanding the unit of analysis (5 points)


### 1.1.1 Print the number of unique values for the following columns all at once (e.g., with `.apply()`), i.e. without copying/pasting code to do each one separately:

- Cases (CASE_ID)
- People (CASE_PARTICIPANT_ID)
- Charges (CHARGE_ID)

**Source for this question**: [slide 30 here on column-wise apply](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf)

In [ ]:
## your code here printing numbers of unique values

### 1.1.2  Cases and people

You might have noticed there are more unique people than unique cases and more unique charges than unique people. This is because the same case can have multiple people involved, and the same person can have multiple charges tied to a case. Illustrate this by showing:
   
- an example of a case involving multiple people
- an example of a person in a case involving multiple charges

**Resources**: groupby and agg covered in:
- [The in-class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_pandas_blank.ipynb) and [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb)

- [These lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf)

In [ ]:
## your code here showing a case with multiple people

In [ ]:
## your code here showing a case with multiple charges

### 1.1.3 Finding mean and median 

- Print the mean and median number of charges per `CASE_PARTICIPANT_ID`
- Print the mean and median number of participants per `CASE_ID`

In [ ]:
## your code here finding mean and median

### 1.1.4 Does the data enable us to follow the same defendant across different cases they're charged in? Write 1 sentence in support of your conclusion.

In [ ]:
## your code here checking for linkage of people across cases

(your text response here)

## 1.2 Which offense is final? (3 points)

First, read the data documentation ([link here](https://datacatalog.cookcountyil.gov/api/views/tg8v-tm6u/files/8597cdda-f7e1-44d1-b0ce-0a4e43f8c980?download=true&filename=CCSAO%20Data%20Glossary.pdf)) and summarize in your own words the differences between `OFFENSE_CATEGORY` and `UPDATED_OFFENSE_CATEGORY`.

(your text response here summarizing the differences)

Then construct an indicator `is_changed_offense` that's True for case-participant-charge observations (rows) where there's a difference between the `OFFENSE_CATEGORY` and the `UPDATED_OFFENSE_CATEGORY`. 

**Resources**: row subsetting, groupby/agg, and np.where covered in [lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf)

In [ ]:
## your code here constructing indicator

What are some of the more common changed offenses? Consider both:
  - The raw number of changed offenses that come from each `OFFENSE_CATEGORY` (e.g., using `value_counts()`). This should answer the question: What offenses contribute the most to the pool of changed offenses?
  - The proportion of each `OFFENSE_CATEGORY` that gets changed (can just compute mean and print result of `sort_values()`). This should answer the question: What offenses tend to get changed the most?

In [ ]:
## your code here inspecting most common changed offenses

Print one example of a changed offense from one of these categories and comment on what the reason may be.

In [ ]:
## your code here printing example

## 1.3 Simplifying the charges (5 points)

Using the field (`UPDATED_OFFENSE_CATEGORY`), create a new field, `simplified_offense_derived`, that simplifies the many offense categories into broader buckets using the following process:

First, create a new variable that strips "Aggravated" (capitalized) from the `UPDATED_OFFENSE_CATEGORY` (e.g., 'Aggravated Battery' just becomes 'Battery', 'Aggravated DUI' becomes 'DUI')

**Resources**: slide 35 of [the lecture on data wrangling with pandas](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf) has str.replace (example with stripping the name "Johnson" from a last name)

In [ ]:
## your code here stripping 'Aggravated'

Then:
- Combine all offenses with 'Arson' in the string into a single `Arson` category
- Combine all offenses with 'Homicide' in the string into a single `Homicide` category
- Combine all offenses with 'Vehic' in the string into a single `Vehicle-related` category
- Combine all offenses with 'Battery' in the string into a single `Battery` category
- Use the simplified offense variable created above (the one without 'Aggravated') as the fallback/default value (instead of 'other')

Do so efficiently, using `np.select()` (or a similar procedure for systematic recoding) rather than separate line for each recoded offense.

**Resources**:
- [Activity code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb) and [lecture on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/01_qss20_w23_pandas.pdf) covers `np.select`.

In [ ]:
## your code here combining offenses

Print the difference between the # of unique offenses in the original `UPDATED_OFFENSE_CATEGORY` field and the # of unique offenses in your new `simplified_offense_derived` field. How many and which ones change?

*Hint*: You can turn unique values from a column into a list using `df[col].unique().tolist()` and get the difference between two lists using a list comprehension: `[elem for elem in list1 if elem not in list2]`. 

In [ ]:
## your code here printing differences

## 1.4: Cleaning sentencing date (3 points)
Create `sentenceymd_derived` that's a version of `SENTENCING_DATE` converted to datetime format. Also create a rounded version, `sentenceym_derived`, that's rounded down to the first of the month and the year (e.g., 01-05-2016 and 01-27-2016 each become 01-01-2016). 

*Hints*: All timestamps are midnight so you can strip the timestamp. Before converting, you'll notice that some of the years have been mistranscribed (e.g., 291X or 221X instead of 201X). Programatically fix those (eg 2914 -> 2014). You can use this regex code to clean the dates or write your own pattern: ### first, use regex to clean up the date columns

```python
sentence['tmp_clnsdate'] = [re.sub(r'2[1-9]([0-9]+)', r"20\1", str(date)) 
                            if bool(re.search('\/2[1-9][0-9]+', str(date))) else 
                            str(date) 
                            for date in 
                            sentence.SENTENCE_DATE]
```

Even after cleaning, there will still be some that are after the year 2021 that we'll filter out later.

**Resources**:

- pd.to_datetime() used in [the data wrangling activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb)
- extract the month and year from a datetime object using the dt accessor (similar syntax for year): https://pandas.pydata.org/docs/reference/api/pandas.Series.dt.month.html 


In [ ]:
## your code here that creates datetime version of sentencing date

## 1.5 Subsetting rows to analytic dataset (5 points)

Let's narrow down the above sentencing dataset in a few ways. First, subset to cases where only one participant is charged, since cases with >1 participant might have complications like plea bargains/informing from other participants affecting the sentencing of the focal participant.

In [ ]:
## your code here to limit to one participant

Next, let's go from a participant-case level dataset, where each participant is repeated across charges tied to the case, to a participant-level dataset, where each participant has one charge. To do this, let's subset to a participant's primary charge and their current sentence (`PRIMARY_CHARGE_FLAG` is True and `CURRENT_SENTENCE_FLAG` is True). Double check that this worked by confirming there are no longer multiple charges for the same case-participant.

In [ ]:
## your code here to subset to primary charge and current sentence

Finally, apply these two additional filters: 

- filter out observations where judge is nan or nonsensical (indicated by `is.null` or equal to `FLOOD`)
- subset to sentencing date between `01-01-2012` and `04-05-2021` (inclusive)

In [ ]:
## your code here to apply remaining filters

# 2. Investigating Black vs. White sentencing disparities (30 points total)

## 2.0 Load the cleaned data (1 point)
Use `pd.read_csv` to load the `sentencing_cleaned.csv` data. This file is included in the same zip file you used for part 1 above. You'll notice the data is slightly different from what we used for part 1 above.

In [ ]:
## your code here loading the data

## 2.1. Investigating one type of between-group difference: who reaches the sentencing stage? (8 points)

Calculate the fraction of Black versus White defendants by month and year:

- Denominator is number of unique cases that month
- Numerator for black defendants is count of `is_black_derived`
- Numerator for white defendants is count of `is_white_derived`
- Fraction of each is numerator/denominator

**Hint:**
For this and other time-based questions in this pset, you can use either `sentenceymd_derived` or `sentenceym_derived` (whichever makes more sense for the question). As a reminder, `sentenceymd_derived` is a version of `SENTENCING_DATE` converted to datetime format, and `sentenceym_derived` is a version rounded down to the first of the month and the year (e.g., 01-05-2016 and 01-27-2016 each become 01-01-2016).

**Concepts tested and resources**:
- Groupby and agg, as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/01_qss20_w23_pandas.pdf), [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_pandas_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb), and [the DataCamp on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas)

- List comprehension (one option), as covered in [these lecture slides on lists & functions](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/02_qss20_w23_pythonbasics.pdf)

In [ ]:
# your code here

- With that calculation, create a graph with two lines: one for Black defendants as fraction of total; another for White defendants. Make sure it includes a legend summarizing which color is for which group, and clean the legend so that it has informative names (e.g., Black or White rather than prop_black or prop_white).
- Use mathematical notation to write out each of the proportions using summation notation in a 1-2 sentence writeup describing trends. What seems to be going on in April and May 2020? 

**Optional challenge** (no extra credit points): improve the viz by shading the background of the visualization for months with fewer than 100 cases 

**Optional challenge** (no extra credit points): improve the viz by adding a vertical line for 12-01-2016, the month that new State's Attorney Foxx took office 

**Hints:**

- Access mathematical notation in Jupyter notebooks with the dollar sign (`$`) special character and commands like `\dfrac{Numerator}{Denominator}` and `\sum_{start}^{end}`, e.g.: 
$\dfrac{\sum_{i}^{N} One thing}{\sum_{i}^{N} Another thing}$

**Concepts tested and resources:** 

- Visualization, as covered in [this plotnine example code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_plotting_examples_plotnine.ipynb), chapter 4 of [the DataCamp course on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas), and the optional DataCamp courses on Data Visualization with ggplot2/Matplotlib

In [ ]:
# your code here

(Your interpretation here)

## 2.2.0 Investigating mechanisms: incarceration rates by charge (21 points total)

Your colleague sees the previous graph and is worried that the gap could be different---either wider or smaller---if you adjust for the fact that prosecutors have discretion in what crimes to charge defendants with. If white defendants are charged with crimes that tend to receive probation rather than incarceration, that could explain some of the gaps.

In the next questions, you'll begin to investigate this.

### 2.2.1 Find the most common offenses (3 points)

First, based on `simplified_offense_derived`, create a set of 'frequent offenses' that represent (over the entire period) the union of the 10 offenses Black defendant are most likely to be charged with and the 10 offenses white defendants are most likely to be charged with (might be far less than 20 total if there's a lot of overlap in common charges)

**Hint:** To get the unique elements of a list (i.e., remove overlaps), create a `set()`, which only stores unique values (syntax slightly different than with lists).

**Concepts tested and resources:**

- Row subsetting and sorting, as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf), [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_pandas_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb), and [the DataCamp on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas)

In [ ]:
# your code here

### 2.2.2 Look at incarceration rates (whether incarcerated) by race and offense type for these top offenses (3 points)

Print a wide-format version of the resulting table (so each row is an offense type, one column is black incarceration rate for that offense type, and another column is the white incarceration rate) and interpret. What offenses show the largest disparities in judges being less likely to sentence White defendants to incarceration/more likely to offer those defendants probation?

According to the codebook, incarceration is indicated by `COMMITMENT_TYPE` == "Illinois Department of Corrections".

**Hint:** To create a wide-format version of a table, one option is [`pd.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html). 

**Concepts tested and resources:**

- Recoding columns using logical conditions (e.g., with np.where) and groupby with agg, as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf) and [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_pandas_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb)

In [ ]:
# your code here

(Your text response here)

### 2.2.3 Examine whether this changes pre and post change to charging threshold for retail theft (10 points)

One important question is not only whether there are disparities by offense type, but also whether these disparities have changed over time.

For instance, the SAO (State Attorney Office) announced in December of 2016 that they would no longer default to charging retail thefts of under \$1,000 as felonies. This change might have (1) decreased disparities or (2) increased disparities, depending on the correlation between race/ethnicity and magnitude of goods stolen (see [this article](https://www.dnainfo.com/chicago/20161215/little-village/kim-foxx-raises-bar-for-retail-theft-felonies/) for more background). 

Focusing on `simplified_offense_derived` == "Retail theft", write a user-defined function that allows you to efficiently: 

- Compare Black-White disparities before and after the change using a two-month bandwidth (so pre is October and November 2016; post is January and February 2017)

- Compare Black-White disparities before and after the change using a four-month bandwidth (so pre is August- November 2016; post is January - April 2017)

- Compare Black-White disparities using an eight-month bandwidth

- Compare Black-White disparities using a twelve-month bandwidth

In other words, the function should compare percentages of defendants incarcerated for retail theft by race. The numerator in the proportions is the # of defendants incarcerated for retail theft, the denom is # of total defendants for retail theft (calculate this separately for each race and separately for before versus after); disparity is the difference in proportions.

Exclude Dec. 2016 as a transition month.

------------------ 

Print a table with the results (any organization is fine as long as it's clear).
 
------------------ 
 
**Concepts tested and resources**:
    
- User-defined functions and list comprehensions, as will be covered in class on 01/18 in [these slides](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/04_qss20_w23_userdefinedfunctions.pdf) and [the functions activity](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/02_functions_blank.ipynb) and [its solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/02_functions_solutions.ipynb) (forthcoming on 01/18)

- Row subsetting with logical conditions (e.g., with np.where), as covered in [these lecture slides on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/slides/03_qss20_w23_pandas.pdf) and [the class activity on data wrangling](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_pandas_blank.ipynb) and its [solutions](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/solutions/01_pandas_solutions.ipynb)

- Visualization, as covered in [this plotnine example code](https://github.com/jhaber-zz/QSS20_public/blob/main/activities/01_plotting_examples_plotnine.ipynb), chapter 4 of [the DataCamp course on Data Manipulation with Pandas](https://app.datacamp.com/learn/courses/data-manipulation-with-pandas), and the optional DataCamp courses on Data Visualization with ggplot2/Matplotlib

**Hints on function:** Your function should take these steps:

1. Create a December 2016 object and use that to create a range corresponding to the bandwidth.
2. Filter to rows within December 2016 +- that # of months. For example, for the 2-month bandwidth, the "before" period is Oct and Nov 2016; after is Jan and Feb 2017. Instead of using timedelta to increment in time (as we did in class function activity), use [**relativedelta**](https://dateutil.readthedocs.io/en/stable/relativedelta.html) to increment in months.
3. Within that filtered dataset, examine Black-White disparities in incarceration before versus after. One shortcut for doing this is to keep the full dataframe together and construct an `is_after` indicator (e.g., using `np.where()`) that takes the value of `True` if after Dec 2016 (and otherwise is `False`), and then group by that and a categorical race variable. This step produces a single dataframe for each time window--e.g., a dataframe for the 2-month bandwidth, a dataframe for the 4-month one, etc. 
4. Use `pd.concat` to combine those dataframes into a single dataframe.

**Hint on output:** The table you make should have two Black-white disparities per bandwidth: one disparity (e.g., a 10 percentage point difference in incarceration rates) before the policy change and another disparity after (e.g., a 5 percentage point difference in incarceration rates). 

In [ ]:
# your code here

### 2.2.4 Visualize the above (4 points)

Use the table you just made to create a bar chart, where the x axis represents different bandwidths (2, 4, etc); the y axis the size of the Black-White gap, and for each of the x axis points, you have one shaded bar representing "before" the change, another representing "after" the change (make sure that before is ordered before after and the bandwidths are from smallest to largest)


**Hints**: 

- For each of the bandwidths, include dates spanning the entire month: e.g., for the first, include not only 02-01-2017 but everything up through 02-28-2017; easiest way is for the subsetting to use the rounded `sentenceym_derived`. Also make sure to only include white or black defendants.

- Depending on how you calculate/reshape things, you may find [this issue useful for how to collapse column names with a multilevel index](https://stackoverflow.com/questions/24290297/pandas-dataframe-with-multiindex-column-merge-levels) (also may not need it depending on how you structure the code).

- The x-axis on the plot should be a categorical variable, with each of the bandwidths and with separate bars for before vs. after. If you want to change the order of the categories, [check out the `reorder_categories` function in this SO issue](https://stackoverflow.com/questions/38023881/pandas-change-the-order-of-levels-of-factor-type-object).

**Extra credit** (1 point): because the bandwidths have different sample sizes, a better viz incorporates measures of uncertainty. Add standard errors to the points using the formula: $(\dfrac{p(1-p)}{n})^{0.5}$ where N is the number of cases in each bandwidth period.

In [ ]:
# your code here

### 2.2.5 Interpret the results (1 point)

Write a two-sentence interpretation of the results. What might this show about how people on both sides of the issue---those arguing the policy change will narrow disparities; those arguing the change may widen disparities--could support their claims? 

(your interpretation here)